# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

/app/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [4]:

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="3"; 

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [12]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [13]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length:
        maximum_length = length
    else:
        maximum_length = max([len(l) for l in x])
    padded_sequences = pad_sequences(x,
                                     maxlen=maximum_length,
                                     dtype='int32',
                                     padding='post',
                                     truncating='post',
                                     value=0.0)
    return padded_sequences

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [32]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    # it seems logits are otput from the model after softmax with shape = (number of prediction, vocabulary_size)
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')


def index_to_text(sentence, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    # it seems logits are otput from the model after softmax with shape = (number of prediction, vocabulary_size)
    return ' '.join([index_to_words[ind[0]] for ind in sentence])




`logits_to_text` function loaded.


# Index Word Dictionary
example of index_to_words = {0: '<PAD>', 1: 'the', 2: 'quick', 3: 'a', 4: 'brown', 5: 'fox', 6: 'jumps', 7: 'over', 8: 'lazy', 9: 'dog', 10: 'by', 11: 'jove', 12: 'my', 13: 'study', 14: 'of', 15: 'lexicography', 16: 'won', 17: 'prize', 18: 'this', 19: 'is', 20: 'short', 21: 'sentence'}

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
from keras import Sequential
from keras.layers import LSTM 


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    

    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)



simple_rnn_model.summary()
    
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=40, validation_split=0.2)

simple_rnn_model.save('simple_rnn_model.h5')  # creates a HDF5 file 'my_model.h5'
# simple_rnn_model = load_model('simple_rnn_model.h5')

/app/local/anaconda3/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 21, 64)            16896     
_________________________________________________________________
dense_2 (Dense)              (None, 21, 344)           22360     
Total params: 39,256
Trainable params: 39,256
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 20s 182us/step - loss: 3.5059 - acc: 0.4104 - val_loss: nan - val_acc: 0.4572
Epoch 2/40
110288/110288 [==============================] - 4s 37us/step - loss: 2.4177 - acc: 0.4696 - val_loss: nan - val_acc: 0.4965
Epoch 3/40
110288/110288 [==============================] - 4s 37us/step - loss: 2.1121 - acc: 0.5214 - val_loss: nan - val_acc: 0.5434
Epoch 4/40
110288/110288 [==============================] - 4s 38us/step 

### Sample Peediction 

In [13]:
print("sample 0")
print("predicted  :", logits_to_text(simple_rnn_model.predict(tmp_x[0:1])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[0], french_tokenizer))

print("sample 100")
print("predicted  :", logits_to_text(simple_rnn_model.predict(tmp_x[100:101])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[100], french_tokenizer))

print("sample 1000")
print("predicted  :", logits_to_text(simple_rnn_model.predict(tmp_x[1000:1001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[1000], french_tokenizer))

print("sample 10000")
print("predicted  :", logits_to_text(simple_rnn_model.predict(tmp_x[10000:10001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[10000], french_tokenizer))

sample 0
predicted  : new jersey est parfois calme en l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 100
predicted  : new jersey est généralement calme en juin mais il est généralement généralement en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est généralement calme au cours de l' automne mais il est généralement chaud en avril <PAD> <PAD> <PAD> <PAD>
sample 1000
predicted  : ils aiment les les les les et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 10000
predicted  : elle aime les citrons le pamplemousse et les citrons <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: elle n'aime les fraise

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3       
    
    model = Sequential()
    model.add(
          Embedding(
          input_dim = english_vocab_size,
          output_dim = 64,
          input_length = input_shape[1],
          trainable= True))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
     
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    

    return model

tests.test_embed_model(embed_model)


tmp_x_padded = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x_reshaped = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x_padded.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)



embed_rnn_model.summary()
    
embed_rnn_model.fit(tmp_x_padded, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)


embed_rnn_model.save('embed_rnn_model.h5')  # creates a HDF5 file 'my_model.h5'
# embed_rnn_model = load_model('embed_rnn_model.h5')

/app/local/anaconda3/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 64)            12736     
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 64)            33024     
_________________________________________________________________
dense_4 (Dense)              (None, 21, 344)           22360     
Total params: 68,120
Trainable params: 68,120
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 7s 64us/step - loss: 3.7743 - acc: 0.4013 - val_loss: nan - val_acc: 0.4093
Epoch 2/20
110288/110288 [==============================] - 4s 39us/step - loss: 2.7325 - acc: 0.4405 - val_loss: nan - val_acc: 0.4818
Epoch 3/20
110288/110288 [==============================] - 5s 43us/step - loss

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

### Sample Peediction 

In [15]:
print("sample 0")
print("predicted  :", logits_to_text(embed_rnn_model.predict(tmp_x_padded[0:1])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[0], french_tokenizer))

print("sample 100")
print("predicted  :", logits_to_text(embed_rnn_model.predict(tmp_x_padded[100:101])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[100], french_tokenizer))

print("sample 1000")
print("predicted  :", logits_to_text(embed_rnn_model.predict(tmp_x_padded[1000:1001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[1000], french_tokenizer))

print("sample 10000")
print("predicted  :", logits_to_text(embed_rnn_model.predict(tmp_x_padded[10000:10001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[10000], french_tokenizer))

sample 0
predicted  : new jersey est parfois calme en l' et il il il enneigée en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 100
predicted  : new jersey est généralement calme en l' automne mais mais il est est chaud en avril <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est généralement calme au cours de l' automne mais il est généralement chaud en avril <PAD> <PAD> <PAD> <PAD>
sample 1000
predicted  : ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 10000
predicted  : elle déteste les fraises le pamplemousse et les raisins <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: elle n'aime les fraise

### Bidirectional LSTM Model 

In [16]:
from keras.layers import Bidirectional

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3    
    
    
    # model form 1
    model = Sequential()
    model.add(
          Embedding(input_dim = english_vocab_size,
          output_dim = 64,
          input_length = input_shape[1],
          trainable= True))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dense(french_vocab_size, activation='softmax'))
    

    
    # model form 2
#     input_seq = Input(shape=input_shape[1:])
#     embedded_seq = Embedding(input_dim = english_vocab_size, 
#                              output_dim = 64,
#                              input_length=input_shape[1])(input_seq)
    
#     rnn = Bidirectional(units=100, return_sequences=True)(embedded_seq)
#     logits = TimeDistributed(Dense(units=french_vocab_size))(rnn) 
#     model = Model(input_seq, Activation('softmax')(logits))    
    
    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    

    return model

tests.test_embed_model(bd_model)


tmp_x_padded = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x_reshaped = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(tmp_x_padded.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)



bd_rnn_model.summary()
    
bd_rnn_model.fit(tmp_x_padded, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

bd_rnn_model.save('bd_rnn_model.h5')  # creates a HDF5 file 'my_model.h5'
# bd_rnn_model = load_model('bd_rnn_model.h5')

/app/local/anaconda3/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 21, 64)            12736     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 128)           66048     
_________________________________________________________________
dense_6 (Dense)              (None, 21, 344)           44376     
Total params: 123,160
Trainable params: 123,160
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 9s 78us/step - loss: 3.4496 - acc: 0.4300 - val_loss: nan - val_acc: 0.4762
Epoch 2/20
110288/110288 [==============================] - 7s 63us/step - loss: 2.3391 - acc: 0.4926 - val_loss: nan - val_acc: 0.5080
Epoch 3/20
110288/110288 [==============================] - 7s 63us/step - lo

### Sample Peediction 

In [17]:
print("sample 0")
print("predicted  :", logits_to_text(bd_rnn_model.predict(tmp_x_padded[0:1])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[0:1][0], french_tokenizer))

print("sample 100")
print("predicted  :", logits_to_text(bd_rnn_model.predict(tmp_x_padded[100:101])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[100:101][0], french_tokenizer))

print("sample 1000")
print("predicted  :", logits_to_text(bd_rnn_model.predict(tmp_x_padded[1000:1001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[1000:1001][0], french_tokenizer))

print("sample 10000")
print("predicted  :", logits_to_text(bd_rnn_model.predict(tmp_x_padded[10000:10001])[0], french_tokenizer))
print("groundtruth:", index_to_text(preproc_french_sentences[10000:10001][0], french_tokenizer))

sample 0
predicted  : new jersey est parfois calme pendant l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 100
predicted  : new jersey est généralement calme au cours de l' automne mais est est habituellement en avril avril <PAD> <PAD> <PAD> <PAD>
groundtruth: new jersey est généralement calme au cours de l' automne mais il est généralement chaud en avril <PAD> <PAD> <PAD> <PAD>
sample 1000
predicted  : ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
sample 10000
predicted  : elle n'aime les fraises le pamplemousse et les raisins <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
groundtruth: elle n'aime 

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

# Custome Model Description
1. `Context`:
For my custome model, i tried to follow the material in the lecture and [encoder decoder models with Keras](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html). The tutorial introduces a encoder-decoder model at the character level, and several modifications was needed to convert it to word level, which i implemented. It took me considerable amount of time to finalize the work. 

2. `Model` : This seq-to-seq model below has the ability to generate predictions of variables length. Which makes it different from the previous models above. I create 2 new tokens in the target sentences (French). One that marks the <start> and the other <end> of the sentence. The graph below shows the structure of the model.

![alt text](images/seq2seq.PNG)


3. `Challenges`: I had to create new tokens <end> , <start> which resulted in having to redefine some of the earlier functions for preprocessing the data. The preprocessing was different from other models. The model framework deviated from the structure defined in the note book: defining a single model in a function and then calling the predict method. I had to create a couple of smaller models from the seq2seq to sequentially and token by token generate my predictions. As a result, i wrote my own validation function. The function at the end of the notebook shows you the predictions vs the actual french sentences. 


### Preprocessing Data

In [9]:
def preprocess_seq2seq(x, y):
    """
    compared to previous version preprocess(x, y) i just removed the part where we add 1 dimension to y
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize_seq2seq(x)
    preprocess_y, y_tk = tokenize_seq2seq(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
#     preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk


def tokenize_seq2seq(x):
    """
    The only difference between this version of the function and tokenize is that
    i removed <> from filters so i can add <start> and <end> tokens to my target sentences 
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer(char_level=False, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk


def append_start_end(original_sentence_seq):
    """
    adding start and end tokens to the target sentences to initiate and end predictions. 
    """
    output = list()
    for sentence in original_sentence_seq:
        output.append('<start> ' + sentence + ' <end>')
    return output


def index_to_text_seq(sentence, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    # it seems logits are otput from the model after softmax with shape = (number of prediction, vocabulary_size)
    return ' '.join([index_to_words[ind] for ind in sentence])



def validation(test_sentences_index):
    for sentence in test_sentences_index:
        print("predicted  :", index_to_text_seq(predict(preproc_english_sentences_seq2seq[sentence,:])[1:], french_tokenizer_seq2seq))
        print("groundtruth:", index_to_text_seq(decoder_output_sequence[sentence,:], french_tokenizer_seq2seq))
     

def sentence_validation(sentence, english_tokenizer, french_tokenizer):
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence_padded = pad([sentence], max_french_sequence_length_seq2seq)
#     print(sentence_padded[0,:].shape)
    predicted_seq = predict(sentence_padded[0,:])
    translation = index_to_text_seq(predicted_seq[1:-1], french_tokenizer)
    return translation


In [14]:
french_sentences_seq2seq = append_start_end(french_sentences)

preproc_english_sentences_seq2seq, preproc_french_sentences_seq2seq, english_tokenizer_seq2seq, french_tokenizer_seq2seq =\
    preprocess_seq2seq(english_sentences, french_sentences_seq2seq)
    
max_english_sequence_length_seq2seq = preproc_english_sentences_seq2seq.shape[1]
max_french_sequence_length_seq2seq = preproc_french_sentences_seq2seq.shape[1]
english_vocab_size_seq2seq = len(english_tokenizer_seq2seq.word_index)
french_vocab_size_seq2seq = len(french_tokenizer_seq2seq.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length_seq2seq)
print("Max French sentence length:", max_french_sequence_length_seq2seq)
print("English vocabulary size:", english_vocab_size_seq2seq)
print("French vocabulary size:", french_vocab_size_seq2seq)


preproc_english_sentences_seq2seq = pad(preproc_english_sentences_seq2seq, max_french_sequence_length_seq2seq)
# here i am sliding the output sequences to generate the input/output to the decoder during prediction. 
decoder_input_sequence = preproc_french_sentences_seq2seq[:,:-1]
decoder_output_sequence = preproc_french_sentences_seq2seq[:,1:]

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 23
English vocabulary size: 199
French vocabulary size: 346


# Training the seq-to-seq model with LSTM Implementation

In [21]:
from keras import Sequential
from keras.layers import LSTM


learning_rate = 1e-3
latent_dimension = 128
input_shape = preproc_english_sentences_seq2seq.shape
output_sequence_length = decoder_output_sequence.shape
english_vocab_size = english_vocab_size_seq2seq
french_vocab_size = french_vocab_size_seq2seq


encoder_inputs = Input(shape=input_shape[1:])
encoder_embedder = Embedding(input_dim = english_vocab_size,
                             output_dim = latent_dimension,
                             trainable= True)
encoder_embeddings = encoder_embedder(encoder_inputs)
encoder_lstm = LSTM(units = latent_dimension, return_state=True)
encoder_last_state_hidden,encoder_last_state_hidden_again,encoder_last_state_cell = \
                                                        encoder_lstm(encoder_embeddings)

encoder_states = [encoder_last_state_hidden, encoder_last_state_cell]

decoder_inputs = Input(shape=(output_sequence_length[1],))
decoder_embedder = Embedding(input_dim = french_vocab_size,
                             output_dim = latent_dimension,
                             trainable= True)
decoder_embeddings = decoder_embedder(decoder_inputs)
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, decoder_output_last_hidden, decoder_output_last_cell = \
                                decoder_lstm(decoder_embeddings,
                                             initial_state=encoder_states)

decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs_prob = decoder_dense(decoder_outputs)
seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs_prob)


seq2seq_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])


# tests.test_model_final(seq2seq_model)


# seq2seq_model.summary()

seq2seq_model.fit([preproc_english_sentences_seq2seq, decoder_input_sequence],
                 decoder_output_sequence.reshape(*decoder_output_sequence.shape, 1),
                 batch_size=1024,
                 epochs=60,
                 validation_split=0.2)


# seq2seq_model.save('seq2seq_lstm.h5')  # creates a HDF5 file 'my_model.h5'
# seq2seq_model = load_model('seq2seq_lstm.h5')

print('Final Model Loaded')


Train on 110288 samples, validate on 27573 samples
Epoch 1/60
110288/110288 [==============================] - 9s 82us/step - loss: 2.9660 - acc: 0.4497 - val_loss: nan - val_acc: 0.5347
Epoch 2/60
110288/110288 [==============================] - 8s 70us/step - loss: 1.8378 - acc: 0.6027 - val_loss: nan - val_acc: 0.6703
Epoch 3/60
110288/110288 [==============================] - 8s 72us/step - loss: 1.1463 - acc: 0.7003 - val_loss: nan - val_acc: 0.7235
Epoch 4/60
110288/110288 [==============================] - 8s 71us/step - loss: 0.8740 - acc: 0.7357 - val_loss: nan - val_acc: 0.7466
Epoch 5/60
110288/110288 [==============================] - 8s 72us/step - loss: 0.7576 - acc: 0.7571 - val_loss: nan - val_acc: 0.7702
Epoch 6/60
110288/110288 [==============================] - 8s 72us/step - loss: 0.6810 - acc: 0.7786 - val_loss: nan - val_acc: 0.7900
Epoch 7/60
110288/110288 [==============================] - 8s 71us/step - loss: 0.6169 - acc: 0.7977 - val_loss: nan - val_acc: 0.80

110288/110288 [==============================] - 8s 70us/step - loss: 0.0191 - acc: 0.9948 - val_loss: nan - val_acc: 0.9936
Final Model Loaded


# Building Models for Prediction
the most tricky part for me, was creating smaller models that use the seq2seq model i trained in the previous step. The encoder_model model simply generate hidden and cell states from the last cell of the encoder LSTM.
This is then passed to the decoder as initial inputs. 
The decoder_inferences takes the hidden states from the encoder and the decoder inputs (words translated upto now), and generates the next translated word. 

In [22]:
# seq2seq_model = load_model('seq2seq_lstm.h5')
encoder_model = Model(encoder_inputs, [encoder_last_state_hidden, encoder_last_state_cell])

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, decoder_state_out_h, decoder_state_out_c = decoder_lstm(
    decoder_embeddings, initial_state=decoder_states_inputs)
decoder_outputs_prob = decoder_dense(decoder_outputs)

decoder_inferences = Model([decoder_inputs,  decoder_state_input_h, decoder_state_input_c],
                           [decoder_state_out_h, decoder_state_out_c, decoder_outputs_prob])

## Prediction (IMPLEMENTATION)
TO generate predictions we first feed <start> token to the decoder and grab the first translation. I then add the translated token to <start> and pass both of them as decoder input to generate the 2nd translated word. The process continues until i either reacht the <end>, which has an prediction index of 3, or reach a <PAD> which has a prediction index of 0. 

In [23]:
def predict(input_seq):
    input_seq = input_seq.reshape(1, *input_seq.shape)
    output_seq = list()
    target_seq = np.zeros((1, max_french_sequence_length_seq2seq-1), dtype=int)
    target_seq[0, 0] = french_tokenizer_seq2seq.word_index['<start>']
    output_seq.append(french_tokenizer_seq2seq.word_index['<start>'])
    h_encoder, c_encoder = encoder_model.predict(input_seq)
    h, c, probabilities = decoder_inferences.predict([target_seq, h_encoder, c_encoder])
    prediction = int(np.argmax(probabilities[0,0,:]))
    output_seq.append(prediction)
    prediction_counter = 0
    while prediction not in [3,0] and prediction_counter < (max_french_sequence_length_seq2seq-2):
        prediction_counter += 1
        target_seq[0, prediction_counter] = prediction
        h, c, probabilities = decoder_inferences.predict([target_seq, h_encoder, c_encoder])
        prediction = int(np.argmax(probabilities[0,prediction_counter,:]))
        output_seq.append(prediction)
    return output_seq

# Model Passes the test
the class requirements on the two sentences proposed is met and the model performs well. but i did not stop here, and tried a few model models. 

In [24]:
validation([0]) 

predicted  : new jersey est parfois calme pendant l' automne et il est neigeux en avril <end>
groundtruth: new jersey est parfois calme pendant l' automne et il est neigeux en avril <end> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [25]:
sentence = "he saw a old yellow truck"
pred = sentence_validation(sentence, english_tokenizer_seq2seq, french_tokenizer_seq2seq)  
print("model prediction: ", pred)

model prediction:  il a vu un vieux camion jaune


# Custome Seq2Seq with Bidirectional LSTM 
this model did not perform as well as the previous model, but keeping this for my record

In [30]:
from keras import Sequential
from keras.layers import LSTM


learning_rate = 1e-3
latent_dimension = 68
input_shape = preproc_english_sentences_seq2seq.shape
output_sequence_length = decoder_output_sequence.shape
english_vocab_size = english_vocab_size_seq2seq
french_vocab_size = french_vocab_size_seq2seq


encoder_inputs = Input(shape=input_shape[1:])
encoder_embedder = Embedding(input_dim = english_vocab_size,
                             output_dim = latent_dimension,
                             trainable= True)
encoder_embeddings = encoder_embedder(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(units = latent_dimension, return_state=True))
h_f_h_b,h_f, c_f, h_b, c_b = encoder_lstm(encoder_embeddings)

encoder_states = [h_f, h_b]
# encoder_states = [encoder_last_state_hidden]
print(encoder_states[0].shape)
decoder_inputs = Input(shape=(output_sequence_length[1],))
decoder_embedder = Embedding(input_dim = french_vocab_size,
                             output_dim = latent_dimension,
                             trainable= True)
decoder_embeddings = decoder_embedder(decoder_inputs)
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, decoder_output_last_hidden, decoder_output_last_cell = \
                                decoder_lstm(decoder_embeddings,
                                             initial_state=encoder_states)

decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs_prob = decoder_dense(decoder_outputs)
seq2seq_bi_model = Model([encoder_inputs, decoder_inputs], decoder_outputs_prob)


seq2seq_bi_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])


# tests.test_model_final(seq2seq_bi_model)


seq2seq_bi_model.summary(line_length=200)

seq2seq_bi_model.fit([preproc_english_sentences_seq2seq, decoder_input_sequence],
                 decoder_output_sequence.reshape(*decoder_output_sequence.shape, 1),
                 batch_size=1024,
                 epochs=40,
                 validation_split=0.2)

# seq2seq_bi_model.save('seq2seq_bi_lstm.h5')  # creates a HDF5 file 'my_model.h5'
# seq2seq_bi_model = load_model('seq2seq_bi_lstm.h5')

print('Final Model Loaded')
# TODO: Train the final model

(?, 68)
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_9 (InputLayer)                                              (None, 23)                                  0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
input_10 (InputLayer)                                             (None, 22)                                  0                                                                             

110288/110288 [==============================] - 11s 102us/step - loss: 0.2800 - acc: 0.9045 - val_loss: nan - val_acc: 0.9076
Epoch 31/40
110288/110288 [==============================] - 11s 103us/step - loss: 0.2696 - acc: 0.9083 - val_loss: nan - val_acc: 0.9087
Epoch 32/40
110288/110288 [==============================] - 11s 102us/step - loss: 0.2624 - acc: 0.9105 - val_loss: nan - val_acc: 0.9112
Epoch 33/40
110288/110288 [==============================] - 11s 102us/step - loss: 0.2547 - acc: 0.9129 - val_loss: nan - val_acc: 0.9128
Epoch 34/40
110288/110288 [==============================] - 11s 103us/step - loss: 0.2471 - acc: 0.9156 - val_loss: nan - val_acc: 0.9151
Epoch 35/40
110288/110288 [==============================] - 11s 103us/step - loss: 0.2409 - acc: 0.9175 - val_loss: nan - val_acc: 0.9144
Epoch 36/40
110288/110288 [==============================] - 11s 104us/step - loss: 0.2341 - acc: 0.9203 - val_loss: nan - val_acc: 0.9213
Epoch 37/40
110288/110288 [============

/app/local/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_12 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'bidirectional_3/while/Exit_3:0' shape=(?, 68) dtype=float32>, <tf.Tensor 'bidirectional_3/while_1/Exit_3:0' shape=(?, 68) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Final Model Loaded


In [31]:
# seq2seq_bi_model = load_model('seq2seq_bi_lstm.h5')
encoder_model = Model(encoder_inputs, [h_f_h_b ,h_f , c_f, h_b, c_b])

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, decoder_state_out_h, decoder_state_out_c = decoder_lstm(
    decoder_embeddings, initial_state=decoder_states_inputs)
decoder_outputs_prob = decoder_dense(decoder_outputs)

decoder_inferences = Model([decoder_inputs,  decoder_state_input_h, decoder_state_input_c],
                           [decoder_state_out_h, decoder_state_out_c, decoder_outputs_prob])

In [32]:
def predict(input_seq):
    input_seq = input_seq.reshape(1, *input_seq.shape)
    output_seq = list()
    target_seq = np.zeros((1, max_french_sequence_length_seq2seq-1), dtype=int)
    target_seq[0, 0] = french_tokenizer_seq2seq.word_index['<start>']
    output_seq.append(french_tokenizer_seq2seq.word_index['<start>'])
    _, h_encoder, _, c_encoder, _ = encoder_model.predict(input_seq)
    h, c, probabilities = decoder_inferences.predict([target_seq, h_encoder, c_encoder])
    prediction = int(np.argmax(probabilities[0,0,:]))
    output_seq.append(prediction)
    prediction_counter = 0
    while prediction not in [3,0]:
        prediction_counter += 1
        target_seq[0, prediction_counter] = prediction
        h, c, probabilities = decoder_inferences.predict([target_seq, h_encoder, c_encoder])
        prediction = int(np.argmax(probabilities[0,prediction_counter,:]))
        output_seq.append(prediction)
    return output_seq

# Model does not pass 
This model surpringly does not pass the class criteria. I did not fine tune it, and suspect it might perform better with more parameters and dropout. 

In [33]:
sentence = "he saw a old yellow truck"
sentence_validation(sentence, english_tokenizer_seq2seq, french_tokenizer_seq2seq)    

'il a vu un petit camion rouge'

In [35]:
validation([0])   

predicted  : new jersey est parfois calme au cours de l' automne et il est généralement froid en mai <end>
groundtruth: new jersey est parfois calme pendant l' automne et il est neigeux en avril <end> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# LSTM Based - Mimicking Attention 
my mentor shared with me the [encoder_decoder](https://machinelearningmastery.com/encoder-decoder-models-text-summarization-keras/) which for a good part motivated me to try another LSTM based model that does not generate variable length prediction, but has access to the output of the encoder in all of its cells. This mimicks the attention mechanism behavior in which we have access to the context in each cell, but unlike attention, the weights assigned to the context are not a function of the decoder LSTM hidden state. The advantage of this model is that it fits inside the framework of the jupyter notebook, and i can call the predict method directly on it to evaluate its performance. Overall i found sticking to the framework of this notebook difficult, if we wanted to implement a more complex model. 

I exprimented with different initial decoder state, including not having any initial states. I got the best performance with the configuration below, but i cannot justify why. 

todo: i need to figure out how the initial state and the input tensors shapes compatbillity works. I can only feed 2 of the: h_fowrward, c_foward, h_backward, c_backward, into the initial state. 

In [59]:
from keras.layers import Bidirectional, Concatenate, concatenate, Dense, TimeDistributed, RepeatVector

def mimi_att_mod(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3    
    latent_dimension = 128*2
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_embedder = Embedding(input_dim = english_vocab_size,
                                 output_dim = latent_dimension,
                                 trainable= True)
    encoder_embeddings = encoder_embedder(encoder_inputs)
    encoder_lstm = Bidirectional(LSTM(units = latent_dimension, return_state=True))
    h_f_h_b, h_fowrward, c_foward, h_backward, c_backward = encoder_lstm(encoder_embeddings)
    encoder_final_emb = concatenate([h_fowrward, c_foward, h_backward, c_backward])
    encoder_final_state = [h_fowrward, c_foward]
    
    encoding_summary = RepeatVector(output_sequence_length)(encoder_final_emb)
    decoder_lstm_mimi = LSTM(units = latent_dimension, return_sequences=True, return_state=False)
    decoder_lstm_output = decoder_lstm_mimi(encoding_summary, initial_state=encoder_final_state)
    decoder_output_mimi = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder_lstm_output)

    mimi_att_model = Model(encoder_inputs, decoder_output_mimi)
    
    mimi_att_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])


    return mimi_att_model

tests.test_embed_model(mimi_att_mod)


tmp_x_padded = pad(preproc_english_sentences, preproc_english_sentences.shape[1])

mimi_att_model = mimi_att_mod(tmp_x_padded.shape,
                    max_french_sequence_length,
                    english_vocab_size,
                    french_vocab_size)


mimi_att_model.summary(line_length=200)
# bd_rnn_model.summary()

mimi_att_model.fit(tmp_x_padded, preproc_french_sentences, batch_size=1024, epochs=40, validation_split=0.2)

# mimi_att_model.save('mimi_att_model.h5')  # creates a HDF5 file 'my_model.h5'
# mimi_att_model = load_model('mimi_att_model.h5')

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_41 (InputLayer)                                             (None, 15)                                  0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
embedding_37 (Embedding)                                          (None, 15, 256)                             50944                   input_41[0][0]                                                

110288/110288 [==============================] - 10s 88us/step - loss: 0.0826 - acc: 0.9773 - val_loss: nan - val_acc: 0.9736
Epoch 27/40
110288/110288 [==============================] - 10s 89us/step - loss: 0.0768 - acc: 0.9790 - val_loss: nan - val_acc: 0.9726
Epoch 28/40
110288/110288 [==============================] - 10s 87us/step - loss: 0.0756 - acc: 0.9793 - val_loss: nan - val_acc: 0.9678
Epoch 29/40
110288/110288 [==============================] - 9s 86us/step - loss: 0.0726 - acc: 0.9799 - val_loss: nan - val_acc: 0.9743
Epoch 30/40
110288/110288 [==============================] - 9s 86us/step - loss: 0.0667 - acc: 0.9816 - val_loss: nan - val_acc: 0.9752
Epoch 31/40
110288/110288 [==============================] - 9s 86us/step - loss: 0.0626 - acc: 0.9827 - val_loss: nan - val_acc: 0.9753
Epoch 32/40
110288/110288 [==============================] - 10s 90us/step - loss: 0.0567 - acc: 0.9844 - val_loss: nan - val_acc: 0.9760
Epoch 33/40
110288/110288 [======================

In [61]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = mimi_att_model
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [ ]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?